In [1]:
import requests, random, tqdm
import pandas as pd
from bs4 import BeautifulSoup
random.seed(42)

In [2]:
df = pd.read_excel('DataScientists.xls')
print('old', df.shape)
df = df[df.dblp.duplicated(keep='last')].sort_values(by=['name']).reset_index(drop=True)
print('new', df.shape)
df.head()

old (1220, 5)
new (141, 5)


,name,country,institution,dblp,expertise
0,abolfazl asudeh,united states,university of illinois at chicago,https://dblp.org/pid/04/7892.html,NaN
1,alfons kemper,germany,technical university of munich,https://dblp.org/pid/k/AlfonsKemper.html,NaN
2,alon halevy,united states,facebook,https://dblp.uni-trier.de/pers/h/Halevy:Alon_Y...,NaN
3,amit awekar,india,iit g,https://dblp.org/pid/136/7882.html,NaN
4,anirban dasgupta,india,iit gandhinagar,https://dblp.org/pid/54/385-1.html,NaN


In [3]:
def get_links(node_url):
    '''node_url: url of node person
       returns: set of people linked to person
    '''
    soup = BeautifulSoup(requests.get(node_url).content, 'html5lib')
    name = [component.text.strip() for component in soup.find('span', attrs={'class': 'name primary'})][0]
    links = {para.text.strip() for para in soup.find_all('span', attrs={'itemprop': 'author'})}
    links.remove(name)
    links = ','.join(sorted(list(links)))
    return dict(name=name, links=links)

In [4]:
clean_df = dict(name=[], country=[], institution=[], links=[], expertise=[])
with tqdm.tqdm(total=len(df)) as pbar:
    for _, row in df.iterrows():
        _, country, institution, node_url, _ = row.values
        data = get_links(node_url)
        clean_df['name'].append(data['name'])
        clean_df['links'].append(data['links'])
        clean_df['country'].append(country)
        clean_df['institution'].append(institution)
        clean_df['expertise'].append(random.randint(1, 10))
        pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [07:37<00:00,  3.25s/it]


In [5]:
clean_df = pd.DataFrame.from_dict(clean_df)
clean_df.head()

,name,country,institution,links,expertise
0,Abolfazl Asudeh,united states,university of illinois at chicago,"Ali Jaoua,Ali Movaghar,Anastasios Sidiropoulos...",2
1,Alfons Kemper,germany,technical university of munich,"Adrian Vogelsgesang,Alex Kulikov,Alexander Bei...",1
2,Alon Y. Halevy,united states,facebook,"Aaron Feng,Abraham Silberschatz,Ada Wai-Chee F...",5
3,Amit Awekar,india,iit g,"Abhinav Anshuman,Abhishek,Abhishek Abhishek,Ak...",4
4,Anirban Dasgupta,india,iit gandhinagar,"Abhimanyu Das,Achint Oommen Thomas,Alexander J...",4


In [6]:
clean_df.to_csv('clean.csv', index=False)